# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
import numpy as np
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files("./data")

In [5]:
# tim.o.> get song data files
for f in get_files("./data/song_data"):
    print(f)

/home/workspace/data/song_data/A/B/B/TRABBLU128F93349CF.json
/home/workspace/data/song_data/A/B/B/TRABBOR128F4286200.json
/home/workspace/data/song_data/A/B/B/TRABBOP128F931B50D.json
/home/workspace/data/song_data/A/B/B/TRABBNP128F932546F.json
/home/workspace/data/song_data/A/B/B/TRABBXU128F92FEF48.json
/home/workspace/data/song_data/A/B/B/TRABBTA128F933D304.json
/home/workspace/data/song_data/A/B/B/TRABBKX128F4285205.json
/home/workspace/data/song_data/A/B/B/TRABBAM128F429D223.json
/home/workspace/data/song_data/A/B/B/TRABBVJ128F92F7EAA.json
/home/workspace/data/song_data/A/B/B/TRABBJE12903CDB442.json
/home/workspace/data/song_data/A/B/B/TRABBBV128F42967D7.json
/home/workspace/data/song_data/A/B/B/TRABBZN12903CD9297.json
/home/workspace/data/song_data/A/B/C/TRABCYE128F934CE1D.json
/home/workspace/data/song_data/A/B/C/TRABCRU128F423F449.json
/home/workspace/data/song_data/A/B/C/TRABCEC128F426456E.json
/home/workspace/data/song_data/A/B/C/TRABCTK128F934B224.json
/home/workspace/data/son

In [6]:
filepath = song_files[0]

In [7]:
df = pd.read_json(filepath, lines=True)
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARNNKDK1187B98BBD5,45.80726,Zagreb Croatia,15.9676,Jinx,407.37914,1,SOFNOQK12AB01840FC,Kutt Free (DJ Volume Remix),0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [8]:
song_data = df.iloc[0][['song_id','title','artist_id','year','duration']].values
song_data=list(song_data)
print(song_data)
song_data[3] = song_data[3].astype(int)
song_data[4] = song_data[4].astype(float)
print(type(song_data[0]))
print(type(song_data[1]))
print(type(song_data[2]))
print(type(song_data[3]))
print(type(song_data[4]))
print(type(song_data))

['SOFNOQK12AB01840FC', 'Kutt Free (DJ Volume Remix)', 'ARNNKDK1187B98BBD5', 0, 407.37914000000001]
<class 'str'>
<class 'str'>
<class 'str'>
<class 'numpy.int64'>
<class 'numpy.float64'>
<class 'list'>


#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [8]:
cur.execute("Select * FROM songs LIMIT 1")
colnames = [desc[0] for desc in cur.description]
print(colnames)

cur.execute(song_table_insert, np.asarray(song_data))
conn.commit()

['song_id', 'title', 'artist_id', 'year', 'duration']


In [10]:
cur.execute("Select * FROM songs LIMIT 0")
colnames = [desc[0] for desc in cur.description]
print(colnames)
cur.execute(song_table_insert, ['SOFNOQK12AB01840FC', 'Kutt Free (DJ Volume Remix)', 'ARNNKDK1187B98BBD5', 0, 407.37914000000001])
conn.commit()

['song_id', 'title', 'artist_id', 'year', 'duration']


In [10]:
print(song_data)

['SOFNOQK12AB01840FC', 'Kutt Free (DJ Volume Remix)', 'ARNNKDK1187B98BBD5', 0, 407.37914000000001]


In [11]:
#psycopg2.extensions.register_type(psycopg2.extensions.UNICODE)
#psycopg2.extensions.register_type(psycopg2.extensions.UNICODEARRAY)

cur.execute(song_table_insert, np.asarray(song_data))
conn.commit()

In [8]:
mysongs = df[['song_id','title','artist_id','year','duration']]
mysongs.head()

,song_id,title,artist_id,year,duration
0,SOFNOQK12AB01840FC,Kutt Free (DJ Volume Remix),ARNNKDK1187B98BBD5,0,407.37914


In [6]:
#
# scans all songs data in ./data/song_data
# insert song records to songs table
#
#for fpath in get_files("./data/song_data"):
for fpath in get_files("./data/song_data"):
    print('processing ',fpath)
    
    mysongs = pd.read_json(fpath, lines=True)   
    for i, row in mysongs.iterrows():
        prow = row[['song_id','title','artist_id','year','duration']]
        print(list(prow))
        # try-catch block or with 
        cur.execute(song_table_insert, list(prow))
        conn.commit()

processing  /home/workspace/data/song_data/A/B/B/TRABBLU128F93349CF.json
['SOFNOQK12AB01840FC', 'Kutt Free (DJ Volume Remix)', 'ARNNKDK1187B98BBD5', 0, 407.37914]
processing  /home/workspace/data/song_data/A/B/B/TRABBOR128F4286200.json
['SOBAYLL12A8C138AF9', 'Sono andati? Fingevo di dormire', 'ARDR4AC1187FB371A1', 0, 511.16363]
processing  /home/workspace/data/song_data/A/B/B/TRABBOP128F931B50D.json
['SOFFKZS12AB017F194', 'A Higher Place (Album Version)', 'ARBEBBY1187B9B43DB', 1994, 236.17261]
processing  /home/workspace/data/song_data/A/B/B/TRABBNP128F932546F.json
['SOGVQGJ12AB017F169', 'Ten Tonne', 'AR62SOJ1187FB47BB5', 2005, 337.68444]
processing  /home/workspace/data/song_data/A/B/B/TRABBXU128F92FEF48.json
['SOXILUQ12A58A7C72A', 'Jenny Take a Ride', 'ARP6N5A1187B99D1A3', 2004, 207.43791]
processing  /home/workspace/data/song_data/A/B/B/TRABBTA128F933D304.json
['SOLEYHO12AB0188A85', 'Got My Mojo Workin', 'ARAGB2O1187FB3A161', 0, 338.23302]
processing  /home/workspace/data/song_data/

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [10]:
artist_data =  df.iloc[0][['artist_id','artist_name','artist_location','artist_latitude','artist_longitude']].values
artist_data = list(artist_data)
artist_data

['ARNNKDK1187B98BBD5',
 'Jinx',
 'Zagreb Croatia',
 45.807259999999999,
 15.967600000000001]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [10]:
cur.execute(artist_table_insert, np.asarray(artist_data))
conn.commit()

In [9]:
#
# scans all songs data in ./data/song_data
# insert artist records to artists table
#
for fpath in get_files("./data/song_data"):
    print('processing ',fpath)
    
    mysongs = pd.read_json(fpath, lines=True)   
    for i, row in mysongs.iterrows():
        prow = row[['artist_id','artist_name','artist_location','artist_latitude','artist_longitude']]
        print(list(prow))
        # try-catch block or with 
        try:
            cur.execute(artist_table_insert, np.asarray(prow))
            conn.commit()
        except (Exception, psycopg2.Error) as error :
            if(conn):
                print("Failed to insert record into artist table", error)

processing  /home/workspace/data/song_data/A/B/B/TRABBLU128F93349CF.json
['ARNNKDK1187B98BBD5', 'Jinx', 'Zagreb Croatia', 45.80726, 15.967600000000001]
Failed to insert record into mobile table current transaction is aborted, commands ignored until end of transaction block

processing  /home/workspace/data/song_data/A/B/B/TRABBOR128F4286200.json
['ARDR4AC1187FB371A1', 'Montserrat Caballé;Placido Domingo;Vicente Sardinero;Judith Blegen;Sherrill Milnes;Georg Solti', '', nan, nan]
Failed to insert record into mobile table current transaction is aborted, commands ignored until end of transaction block

processing  /home/workspace/data/song_data/A/B/B/TRABBOP128F931B50D.json
['ARBEBBY1187B9B43DB', 'Tom Petty', 'Gainesville, FL', nan, nan]
Failed to insert record into mobile table current transaction is aborted, commands ignored until end of transaction block

processing  /home/workspace/data/song_data/A/B/B/TRABBNP128F932546F.json
['AR62SOJ1187FB47BB5', 'Chase & Status', '', nan, nan]
Faile

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [10]:
log_files = get_files("./data/log_data")

In [11]:
filepath = log_files[0]

#for f in log_files: 
#    print(f)

In [12]:
df = pd.read_json(filepath, lines=True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Sydney Youngblood,Logged In,Jacob,M,53,Klein,238.07955,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,Ain't No Sunshine,200,1543449657796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
1,Gang Starr,Logged In,Layla,F,88,Griffin,151.92771,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My Advice 2 You (Explicit),200,1543449690796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
2,3OH!3,Logged In,Layla,F,89,Griffin,192.52200,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My First Kiss (Feat. Ke$ha) [Album Version],200,1543449841796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
3,RÃÂ¶yksopp,Logged In,Jacob,M,54,Klein,369.81506,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,The Girl and The Robot,200,1543449895796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Kajagoogoo,Logged In,Layla,F,90,Griffin,223.55546,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,Too Shy,200,1543450033796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [7]:
df = df[df['page']=='NextSong']
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Sydney Youngblood,Logged In,Jacob,M,53,Klein,238.07955,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,Ain't No Sunshine,200,1543449657796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
1,Gang Starr,Logged In,Layla,F,88,Griffin,151.92771,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My Advice 2 You (Explicit),200,1543449690796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
2,3OH!3,Logged In,Layla,F,89,Griffin,192.52200,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My First Kiss (Feat. Ke$ha) [Album Version],200,1543449841796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
3,RÃÂ¶yksopp,Logged In,Jacob,M,54,Klein,369.81506,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,The Girl and The Robot,200,1543449895796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Kajagoogoo,Logged In,Layla,F,90,Griffin,223.55546,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,Too Shy,200,1543450033796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24


In [8]:
print(df.shape)
print(df['ts'].shape)
df_test = df['ts']
print(df_test.shape)
print(df_test.index)
print(df.index)

(319, 18)
(319,)
(319,)
Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
            ...
            355, 356, 357, 358, 359, 360, 362, 363, 364, 365],
           dtype='int64', length=319)
Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
            ...
            355, 356, 357, 358, 359, 360, 362, 363, 364, 365],
           dtype='int64', length=319)


In [17]:
import datetime
# fromtimestamp does not handle millisecond, hence convert to seconds
# revise microsecond attribute by adding the milliseconds part of the original timestamp
t = df['ts'].apply(lambda x: datetime.datetime.fromtimestamp(x/1000.0).replace(microsecond=1000*(x%1000)))
t.head()

0   2018-11-29 00:00:57.796
1   2018-11-29 00:01:30.796
2   2018-11-29 00:04:01.796
3   2018-11-29 00:04:55.796
4   2018-11-29 00:07:13.796
Name: ts, dtype: datetime64[ns]

In [21]:
# tim.o.
print(t.index)

Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
            ...
            355, 356, 357, 358, 359, 360, 362, 363, 364, 365],
           dtype='int64', length=319)


In [18]:
import numpy as np
#time_data = (t.apply(lambda x: [x.hour, x.day,int(x.day/7),x.month, x.year,x.weekday()]))
time_data = np.asarray([t.dt.hour, t.dt.day, 0,t.dt.month, t.dt.year, t.dt.weekday])
             
#print(time_data.shape)
#time_data = time_data.reshape((time_data.shape,6))
#time_data = np.asarray(time_data)
#print(time_data.shape)

#print(time_data)
#time_data = np.array([time_data[0], time_data[1]])
#print(time_data.shape)

column_labels = ('start_time','hour','day','week','month','year','weekday')
print(column_labels)
#print(t.dt.hour)
#print(type(t.dt.hour))

# tim.o.> get values from dt['ts'] and index from dt
df_tstamp = pd.DataFrame(data=df['ts'].values,index=df.index,columns=['start_time'])
#print(df_tstamp)

# tim.o.> use original index from t
df_hours = pd.DataFrame(data=np.asarray(t.dt.hour),index=t.index,columns=['hour'])
df_days = pd.DataFrame(data=np.asarray(t.dt.day),index=t.index,columns=['day'])
df_wofyear = pd.DataFrame(data=np.asarray(t.dt.day/7),index=t.index,columns=['week'])
df_month = pd.DataFrame(data=np.asarray(t.dt.month),index=t.index,columns=['month'])
df_year = pd.DataFrame(data=np.asarray(t.dt.year),index=t.index,columns=['year'])
df_weekday = pd.DataFrame(data=np.asarray(t.dt.weekday),index=t.index,columns=['weekday'])

dff = pd.concat([df_tstamp,df_hours, df_days,df_wofyear,df_month,df_year,df_weekday], join='outer', axis=1)
dff

('start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday')


,start_time,hour,day,week,month,year,weekday
0,1543449657796,0,29,4.142857,11,2018,3
1,1543449690796,0,29,4.142857,11,2018,3
2,1543449841796,0,29,4.142857,11,2018,3
3,1543449895796,0,29,4.142857,11,2018,3
4,1543450033796,0,29,4.142857,11,2018,3
5,1543450256796,0,29,4.142857,11,2018,3
6,1543450264796,0,29,4.142857,11,2018,3
7,1543450450796,0,29,4.142857,11,2018,3
8,1543450562796,0,29,4.142857,11,2018,3
9,1543450691796,0,29,4.142857,11,2018,3


In [19]:
time_df = pd.DataFrame(data=dff,columns=column_labels)
time_df['week'] = time_df['week'].apply(lambda x: int(x))
print(time_df.tail())

print(list(time_df.iloc[0]))


        start_time  hour  day  week  month  year  weekday
360  1543532012796    22   29     4     11  2018        3
362  1543532242796    22   29     4     11  2018        3
363  1543532525796    23   29     4     11  2018        3
364  1543532713796    23   29     4     11  2018        3
365  1543533515796    23   29     4     11  2018        3
[1543449657796, 0, 29, 4, 11, 2018, 3]


In [39]:
# tim.o.> test insering one record
cur.execute(time_table_insert, [1543449657, 0, 29, 4, 11, 2018, 3])
conn.commit()

In [20]:
print(type(time_df.iloc[0][0]))
time_df['start_time'] = time_df['start_time'].astype(np.longlong);
print(type(time_df.start_time[0]))

<class 'numpy.int64'>
<class 'numpy.int64'>


In [10]:
# tim.o.> test insering one record
cur.execute(time_table_insert, [1543449657796, 0, 29, 4, 11, 2018, 3])
conn.commit()

In [21]:
# tim.o.> test insering one record
cur.execute('SELECT * from information_schema.columns WHERE TABLE_NAME=\'time\';')
results = cur.fetchone()
print(results)

#conn.commit()

('sparkifydb', 'public', 'time', 'start_time', 1, None, 'YES', 'bigint', None, None, 64, 2, 0, None, None, None, None, None, None, None, None, None, None, None, None, 'sparkifydb', 'pg_catalog', 'int8', None, None, None, None, '1', 'NO', 'NO', None, None, None, None, None, None, 'NEVER', None, 'YES')


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [21]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

In [27]:
import numpy as np
import datetime

def getTimeDF(df):
    # fromtimestamp does not handle millisecond, hence convert to seconds
    # revise microsecond attribute by adding the milliseconds part of the original timestamp
    t = df['ts'].apply(lambda x: datetime.datetime.fromtimestamp(x/1000.0).replace(microsecond=1000*(x%1000)))

    time_data = np.asarray([t.dt.hour, t.dt.day, 0,t.dt.month, t.dt.year, t.dt.weekday])
    column_labels = ('start_time','hour','day','week','month','year','weekday')

    # tim.o.> get values from dt['ts'] and index from dt
    df_tstamp = pd.DataFrame(data=df['ts'].values,index=df.index,columns=['start_time'])
    #print(df_tstamp)

    # tim.o.> use original index from t
    df_hours = pd.DataFrame(data=np.asarray(t.dt.hour),index=t.index,columns=['hour'])
    df_days = pd.DataFrame(data=np.asarray(t.dt.day),index=t.index,columns=['day'])
    df_wofyear = pd.DataFrame(data=np.asarray(t.dt.day/7),index=t.index,columns=['week'])
    df_wofyear['week'] = df_wofyear['week'].apply(lambda x: int(x))
    df_month = pd.DataFrame(data=np.asarray(t.dt.month),index=t.index,columns=['month'])
    df_year = pd.DataFrame(data=np.asarray(t.dt.year),index=t.index,columns=['year'])
    df_weekday = pd.DataFrame(data=np.asarray(t.dt.weekday),index=t.index,columns=['weekday'])

    dff = pd.concat([df_tstamp,df_hours, df_days,df_wofyear,df_month,df_year,df_weekday], join='outer', axis=1)
    #time_df = pd.DataFrame(data=dff,columns=column_labels)
    #time_df['week'] = time_df['week'].apply(lambda x: int(x))
    return dff

# Read 
log_files = get_files("./data/log_data")

# For each file 
for filepath in log_files:
    print('processing ',filepath)
    df = pd.read_json(filepath, lines=True)
    df = df[df['page']=='NextSong']
    time_df = getTimeDF(df) 
    for i, row in time_df.iterrows():
        try:
            #print(list(row))
            cur.execute(time_table_insert, list(row))
            conn.commit()
        except (Exception, psycopg2.Error) as error :
            if(conn):
                print("Failed to insert record into time table", error)
        finally:
            if(conn):
                conn.close()
            # tim.o.> reconnect
            conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
            cur = conn.cursor()

processing  /home/workspace/data/log_data/2018/11/2018-11-29-events.json
processing  /home/workspace/data/log_data/2018/11/2018-11-02-events.json
processing  /home/workspace/data/log_data/2018/11/2018-11-20-events.json
processing  /home/workspace/data/log_data/2018/11/2018-11-30-events.json
processing  /home/workspace/data/log_data/2018/11/2018-11-25-events.json
processing  /home/workspace/data/log_data/2018/11/2018-11-04-events.json
processing  /home/workspace/data/log_data/2018/11/2018-11-28-events.json
processing  /home/workspace/data/log_data/2018/11/2018-11-07-events.json
processing  /home/workspace/data/log_data/2018/11/2018-11-22-events.json
processing  /home/workspace/data/log_data/2018/11/2018-11-13-events.json
processing  /home/workspace/data/log_data/2018/11/2018-11-03-events.json
processing  /home/workspace/data/log_data/2018/11/2018-11-11-events.json
processing  /home/workspace/data/log_data/2018/11/2018-11-12-events.json
processing  /home/workspace/data/log_data/2018/11/2

In [28]:
# Read 
log_files = get_files("./data/log_data")

# For each file 
for filepath in log_files:
    print('processing ',filepath)
    df = pd.read_json(filepath, lines=True)
    df = df[df['page']=='NextSong']
    
    # user data
    user_df = df[['userId','firstName','lastName','gender','level']]
    # tim.o.> test, remove duplicate records
    user_df = user_df.drop_duplicates()
    for i, row in user_df.iterrows():
        try:
            cur.execute(user_table_insert, list(row))
            conn.commit()
        except (Exception, psycopg2.Error) as error :
            if(conn):
                print("Failed to insert record into user table", error)
        finally:
            if(conn):
                conn.close()
            # tim.o.> reconnect
            conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
            cur = conn.cursor()

processing  /home/workspace/data/log_data/2018/11/2018-11-29-events.json
Failed to insert record into user table duplicate key value violates unique constraint "users_pkey"
DETAIL:  Key (user_id)=(73) already exists.

Failed to insert record into user table duplicate key value violates unique constraint "users_pkey"
DETAIL:  Key (user_id)=(24) already exists.

Failed to insert record into user table duplicate key value violates unique constraint "users_pkey"
DETAIL:  Key (user_id)=(50) already exists.

Failed to insert record into user table duplicate key value violates unique constraint "users_pkey"
DETAIL:  Key (user_id)=(54) already exists.

Failed to insert record into user table duplicate key value violates unique constraint "users_pkey"
DETAIL:  Key (user_id)=(32) already exists.

Failed to insert record into user table duplicate key value violates unique constraint "users_pkey"
DETAIL:  Key (user_id)=(4) already exists.

Failed to insert record into user table duplicate key valu

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [22]:
user_df = df[['userId','firstName','lastName','gender','level']]
user_df.head()
#print(user_df[0])

,userId,firstName,lastName,gender,level
0,73,Jacob,Klein,M,paid
1,24,Layla,Griffin,F,paid
2,24,Layla,Griffin,F,paid
3,73,Jacob,Klein,M,paid
4,24,Layla,Griffin,F,paid


In [23]:
# tim.o.> test, remove duplicate records
user_df = user_df.drop_duplicates()
for i, row in user_df.iterrows():
    print(list(row))
    if(row[0]==''):
        print('Empty User id')

['73', 'Jacob', 'Klein', 'M', 'paid']
['24', 'Layla', 'Griffin', 'F', 'paid']
['50', 'Ava', 'Robinson', 'F', 'free']
['54', 'Kaleb', 'Cook', 'M', 'free']
['32', 'Lily', 'Burns', 'F', 'free']
['4', 'Alivia', 'Terrell', 'F', 'free']
['52', 'Theodore', 'Smith', 'M', 'free']
['14', 'Theodore', 'Harris', 'M', 'free']
['98', 'Jordyn', 'Powell', 'F', 'free']
['101', 'Jayden', 'Fox', 'M', 'free']
['78', 'Chloe', 'Roth', 'F', 'free']
['10', 'Sylvie', 'Cruz', 'F', 'free']
['49', 'Chloe', 'Cuevas', 'F', 'paid']
['16', 'Rylan', 'George', 'M', 'paid']
['26', 'Ryan', 'Smith', 'M', 'free']
['79', 'James', 'Martin', 'M', 'free']
['82', 'Avery', 'Martinez', 'F', 'paid']
['44', 'Aleena', 'Kirby', 'F', 'paid']
['75', 'Joseph', 'Gutierrez', 'M', 'free']
['39', 'Walter', 'Frye', 'M', 'free']
['80', 'Tegan', 'Levine', 'F', 'paid']
['55', 'Martin', 'Johnson', 'M', 'free']
['12', 'Austin', 'Rosales', 'M', 'free']
['9', 'Wyatt', 'Scott', 'M', 'free']
['22', 'Sean', 'Wilson', 'F', 'free']
['74', 'Braden', 'Park

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [24]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

In [9]:
# tim.o.> see whether we have the data
#
for index, row in df.iterrows():
    
    #print(row.song, row.artist,row.length)
    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length,))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None
    print(songid, row.song, artistid,row.length)

None Ain't No Sunshine None 238.07955
None My Advice 2 You (Explicit) None 151.92771
None My First Kiss (Feat. Ke$ha) [Album Version] None 192.522
None The Girl and The Robot None 369.81506
None Too Shy None 223.55546
None Potential None 306.54649
None The Great Satan (What Would Satan Do Mix) None 186.69669
None The Screamer None 117.31546
None Knife Going In (Album Version) None 129.67138
None The Greatest View (Album Version) None 242.96444
None T.N.T. For The Brain (112 Bpm) (Radio Edit) None 242.78159
None Mon MÃÂ©tro None 278.12526
None Fly Like An Eagle None 254.06649
None Restless None 307.19955
None You're The One None 239.3073
None Blue None 141.92281
None Channel 10 None 194.82077
None The Oak Tree (Album Version) None 442.46159
None What If I Do? None 302.91546
None You're Not Alone None 264.12363
None En Glad Calypso Om VÃÂ¥ren (Live '62) None 202.39628
None Down_ Down_ Down To Mehphisto's Cafe (Album Version) None 303.33342
None Tu Eres Ajena None 239.67302
None Estadio

In [8]:
# tim.o.> see whether we have the data
#
def checkSongArtist(df):
    cnt = 0
    for index, row in df.iterrows():

        #print(row.song, row.artist,row.length)
        # get songid and artistid from song and artist tables
        cur.execute(song_select, (row.song, row.artist, row.length,))
        results = cur.fetchone()
    
        if results:
            songid, artistid = results
        else:
            songid, artistid = None, None
        #print(songid, row.song, artistid,row.length)
    print('Found',cnt,'matches!')

In [9]:
# Read 
log_files = get_files("./data/log_data")

# For each file 
for filepath in log_files:
    print('processing ',filepath)
    df = pd.read_json(filepath, lines=True)
    df = df[df['page']=='NextSong']
    checkSongArtist(df)

processing  /home/workspace/data/log_data/2018/11/2018-11-29-events.json
Found 0 matches!
processing  /home/workspace/data/log_data/2018/11/2018-11-02-events.json
Found 0 matches!
processing  /home/workspace/data/log_data/2018/11/2018-11-20-events.json
Found 0 matches!
processing  /home/workspace/data/log_data/2018/11/2018-11-30-events.json
Found 0 matches!
processing  /home/workspace/data/log_data/2018/11/2018-11-25-events.json
Found 0 matches!
processing  /home/workspace/data/log_data/2018/11/2018-11-04-events.json
Found 0 matches!
processing  /home/workspace/data/log_data/2018/11/2018-11-28-events.json
Found 0 matches!
processing  /home/workspace/data/log_data/2018/11/2018-11-07-events.json
Found 0 matches!
processing  /home/workspace/data/log_data/2018/11/2018-11-22-events.json
Found 0 matches!
processing  /home/workspace/data/log_data/2018/11/2018-11-13-events.json
Found 0 matches!
processing  /home/workspace/data/log_data/2018/11/2018-11-03-events.json
Found 0 matches!
processing

In [36]:
# tim.o.> check individual records
#
q_song = "Superconfidential"
q_song = "You Belong WIth Me" #"Ain't No Sunshine"
q_length = 207.0722
q_artist = ""

cur.execute('SELECT * FROM songs WHERE title LIKE %s;', (q_song,))
results = cur.fetchone() 
print(results)


None


In [38]:
for index, row in df.iterrows():

    print(row.song, '|', row.artist, '|', row.length)
    # get songid and artistid from song and artist tables
    #cur.execute(song_select, (row.song, row.artist, row.length,))
    cur.execute('SELECT * FROM songs WHERE title LIKE TRIM(%s);', (row.song,))
    results = cur.fetchone()
    
    if results:
        songid, aTitle, artistid = results
        print('Found [',row.song,'] id =[', songid,']')
    else:
        songid, aTitle, artistid = None, None, None
        print('Not Found [',row.song,'] ')
    

Ain't No Sunshine | Sydney Youngblood | 238.07955
Not Found [ Ain't No Sunshine ] 
My Advice 2 You (Explicit) | Gang Starr | 151.92771
Not Found [ My Advice 2 You (Explicit) ] 
My First Kiss (Feat. Ke$ha) [Album Version] | 3OH!3 | 192.522
Not Found [ My First Kiss (Feat. Ke$ha) [Album Version] ] 
The Girl and The Robot | RÃÂ¶yksopp | 369.81506
Not Found [ The Girl and The Robot ] 
Too Shy | Kajagoogoo | 223.55546
Not Found [ Too Shy ] 
Potential | The Jimmy Castor Bunch | 306.54649
Not Found [ Potential ] 
The Great Satan (What Would Satan Do Mix) | Ministry | 186.69669
Not Found [ The Great Satan (What Would Satan Do Mix) ] 
The Screamer | Bouncing Souls | 117.31546
Not Found [ The Screamer ] 
Knife Going In (Album Version) | Tegan And Sara | 129.67138
Not Found [ Knife Going In (Album Version) ] 
The Greatest View (Album Version) | Silverchair | 242.96444
Not Found [ The Greatest View (Album Version) ] 
T.N.T. For The Brain (112 Bpm) (Radio Edit) | Enigma | 242.78159
Not Found [ T.N

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [16]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length,))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    # INSERT INTO songplays (start_time, user_id, level, song_id, artist_id, session_id, location, user_agent)
    # VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
    songplay_data = (row.ts, row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent,)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

InterfaceError: cursor already closed

In [4]:
def insertSongplay(df,cnt):
    for index, row in df.iterrows():

        # get songid and artistid from song and artist tables
        cur.execute(song_select, (row.song, row.artist, row.length,))
        results = cur.fetchone()
    
        if results:
            songid, artistid = results
        else:
            songid, artistid = None, None

        # insert songplay record
        cnt += 1
        # INSERT INTO songplays (start_time, user_id, level, song_id, artist_id, session_id, location, user_agent)
        # VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        songplay_data = (cnt, row.ts, row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent,)
        cur.execute(songplay_table_insert, songplay_data)
        conn.commit()
        
    return cnt

In [5]:
# Read 
log_files = get_files("./data/log_data")
cnt = 1
# For each file 
for filepath in log_files:
    print('processing ',filepath)
    df = pd.read_json(filepath, lines=True)
    df = df[df['page']=='NextSong']
    # get cnt as select max(songplay_id) from songplays
    cnt = insertSongplay(df,cnt)

processing  /home/workspace/data/log_data/2018/11/2018-11-29-events.json
processing  /home/workspace/data/log_data/2018/11/2018-11-02-events.json
processing  /home/workspace/data/log_data/2018/11/2018-11-20-events.json
processing  /home/workspace/data/log_data/2018/11/2018-11-30-events.json
processing  /home/workspace/data/log_data/2018/11/2018-11-25-events.json
processing  /home/workspace/data/log_data/2018/11/2018-11-04-events.json
processing  /home/workspace/data/log_data/2018/11/2018-11-28-events.json
processing  /home/workspace/data/log_data/2018/11/2018-11-07-events.json
processing  /home/workspace/data/log_data/2018/11/2018-11-22-events.json
processing  /home/workspace/data/log_data/2018/11/2018-11-13-events.json
processing  /home/workspace/data/log_data/2018/11/2018-11-03-events.json
processing  /home/workspace/data/log_data/2018/11/2018-11-11-events.json
processing  /home/workspace/data/log_data/2018/11/2018-11-12-events.json
processing  /home/workspace/data/log_data/2018/11/2

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [6]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.